In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/harcascade/haarcascade_frontalface_default.xml
/kaggle/input/test-images/0214.jpg
/kaggle/input/test-images/0213.jpg
/kaggle/input/test-images/0131.jpg
/kaggle/input/test-images/0216.jpg
/kaggle/input/test-images/0473.jpg
/kaggle/input/test-images/0215.jpg
/kaggle/input/test-images/0054.jpg
/kaggle/input/resnet-model2/resnet50_mymodel (1).h5
/kaggle/input/not-engaged/0074.jpg
/kaggle/input/not-engaged/0077.jpg
/kaggle/input/not-engaged/0203.jpg
/kaggle/input/not-engaged/0070.jpg
/kaggle/input/not-engaged/0078.jpg
/kaggle/input/not-engaged/0082.jpg
/kaggle/input/not-engaged/0067.jpg
/kaggle/input/not-engaged/0076.jpg
/kaggle/input/not-engaged/0021.jpg
/kaggle/input/not-engaged/0065.jpg
/kaggle/input/not-engaged/0079.jpg
/kaggle/input/not-engaged/0023.jpg
/kaggle/input/not-engaged/0068.jpg
/kaggle/input/not-engaged/0022.jpg
/kaggle/input/not-engaged/0102.jpg
/kaggle/input/not-engaged/0075.jpg
/kaggle/input/not-engaged/0073.jpg
/kaggle/input/not-engaged/0083.jpg
/kaggle/inpu

In [2]:
import os
import cv2
import sys
import dlib
import argparse
import numpy as np
import pandas as pd
# %pylab inline
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

In [3]:
import seaborn as sn
import skimage.io
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

2021-09-28 06:08:12.006648: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-28 06:08:12.006763: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
def drawPolyline(img, shapes, start, end, isClosed=False):
    points = []
    for i in range(start, end + 1):
        point = [shapes.part(i).x, shapes.part(i).y]
        points.append(point)
    points = np.array(points, dtype=np.int32)
    cv2.polylines(img, [points], isClosed, (255, 80, 0),
                  thickness=1, lineType=cv2.LINE_8)

def draw(img, shapes):
    drawPolyline(img, shapes, 0, 16)
    drawPolyline(img, shapes, 17, 21)
    drawPolyline(img, shapes, 22, 26)
    drawPolyline(img, shapes, 27, 30)
    drawPolyline(img, shapes, 30, 35, True)
    drawPolyline(img, shapes, 36, 41, True)
    drawPolyline(img, shapes, 42, 47, True)
    drawPolyline(img, shapes, 48, 59, True)
    drawPolyline(img, shapes, 60, 67, True)

In [5]:
def ref3DModel():
    modelPoints = [[0.0, 0.0, 0.0],
                   [0.0, -330.0, -65.0],
                   [-225.0, 170.0, -135.0],
                   [225.0, 170.0, -135.0],
                   [-150.0, -150.0, -125.0],
                   [150.0, -150.0, -125.0]]
    return np.array(modelPoints, dtype=np.float64)


def ref2dImagePoints(shape):
    imagePoints = [[shape.part(30).x, shape.part(30).y],
                   [shape.part(8).x, shape.part(8).y],
                   [shape.part(36).x, shape.part(36).y],
                   [shape.part(45).x, shape.part(45).y],
                   [shape.part(48).x, shape.part(48).y],
                   [shape.part(54).x, shape.part(54).y]]
    return np.array(imagePoints, dtype=np.float64)


def cameraMatrix(h,w):
    mat = [[1100, 1, h],
           [0, 1100, w],
           [0, 0, 1]]
    return np.array(mat, dtype=float)

In [6]:
PREDICTOR_PATH = os.path.join('../input/shape-predictor-68-face-landmarksdhyani/shape_predictor_68_face_landmarks.dat')


In [7]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../input/shape-predictor-68-face-landmarksdhyani/shape_predictor_68_face_landmarks.dat')
def headposition(filepath):
    
    
    

    im = cv2.imread(filepath)

    faces = detector(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), 0)
    face3Dmodel = ref3DModel()


    for face in faces:
        shape = predictor(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), face)
        draw(im, shape)
        refImgPts = ref2dImagePoints(shape)
        height, width, channel = im.shape





        cameraMatrix1 = cameraMatrix(height/2,width/2)
        mdists = np.zeros((4, 1), dtype=np.float64)


        # calculate rotation and translation vector using solvePnP
        success, rotationVector, translationVector = cv2.solvePnP(face3Dmodel, refImgPts, cameraMatrix1, mdists)


        noseEndPoints3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
        noseEndPoint2D, jacobian = cv2.projectPoints(
        noseEndPoints3D, rotationVector, translationVector, cameraMatrix1, mdists)




        # calculating angle
        rmat, jac = cv2.Rodrigues(rotationVector)
        angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

        print("Angle: ", angles)
        #print(f"Qx:{Qx}\tQy:{Qy}\tQz:{Qz}\t")
        x = np.arctan2(Qx[2][1], Qx[2][2])
        y = np.arctan2(-Qy[2][0], np.sqrt((Qy[2][1] * Qy[2][1] ) + (Qy[2][2] * Qy[2][2])))
        z = np.arctan2(Qz[0][0], Qz[1][0])
#         print("AxisX: ", x)
#         print("AxisY: ", y)
#         print("AxisZ: ", z)
#         print('*' * 80)

        gaze = False        
        if angles[1] < -5:
            gaze = True         
        elif angles[1] > 5:
            gaze = True
        elif angles[0]<-173 :
            gaze =True
        else:
            gaze = False
            
        if(gaze==True):
            return 0
        else:
            return 1
        
        
        


    

In [8]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [9]:

def prepare_j(filepath,face_cas="../input/harcascade/haarcascade_frontalface_default.xml"):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    resized_array = cv2.resize(img_array, (224, 224))
    return resized_array.reshape(-1, 224, 224, 3)
model_j = tf.keras.models.load_model('../input/resnet-model2/resnet50_mymodel (1).h5',custom_objects={'f1_score': f1_score})
model_j.summary()

2021-09-28 06:08:18.920493: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-28 06:08:18.923415: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-28 06:08:18.923445: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-28 06:08:18.923469: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (453b3404e14c): /proc/driver/nvidia/version does not exist
2021-09-28 06:08:18.924576: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 2048)        8192      
_________________________________________________________________
activation (Activation)      (None, 7, 7, 2048)        0         
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 2048)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 7, 128)         6553728   
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 128)         0

In [10]:
labels_new = ["yawn", "no_yawn", "Closed", "Open"]
IMG_SIZE = 145
def prepare(filepath, face_cas="../input/harcascade/haarcascade_frontalface_default.xml"):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return resized_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

model_d = tf.keras.models.load_model("../input/drowsiness/drowiness_new_finalmee.h5")

In [11]:
my_list1=[]
drow=[]
file_path=[]
emotion=[]
label=[]
for subdir, dirs, files in os.walk('../input/studentengagement/Student-engagement-dataset/Engaged/frustrated/'):
    cnt=0
    for file in files:
        filepath = subdir +  file
        print(filepath)
        file_path.append(filepath)
        prediction = model_d.predict([prepare(filepath)])
        prediction2 = model_j.predict([prepare_j(filepath)])
        if(np.argmax(prediction)==2 or np.argmax(prediction)==0):
            status1 = "Drowsy!!"
        else:
            status1 = "Active!!"
        my_list1.append(headposition(filepath))
        drow.append(status1)
        if(np.argmax(prediction2)==0 or np.argmax(prediction2)==2):
            status2 = "Frustrated"
        elif (np.argmax(prediction2)==1 or np.argmax(prediction2)==6):
            status2 = "Confused"
        elif (np.argmax(prediction2)==3 or np.argmax(prediction2)==4):
            status2 = "Engaged"
        else:
            status2 = "Bored"
        emotion.append(status2) 
        point=0
        if(status1=="Drowsy!!"):
            status_final=0
            label.append(status_final)
        else:
            if(headposition(filepath)==1):
                point+=0.15

            if(status2 == "Engaged"):
                point+=0.85
            elif(status2 == "Confused"):
                 point+=0.5
            elif(status2=="Frustrated"):
                 point+=0.25
            else:
                 point+=0
            label.append(point)        

          
         
        
       
        
        
        
        
        


../input/studentengagement/Student-engagement-dataset/Engaged/frustrated/0106.jpg


2021-09-28 06:08:24.549579: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-28 06:08:24.564882: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


Angle:  (-163.05745722191273, 2.5698992379951453, 2.274617397275522)
Angle:  (-163.05745722191273, 2.5698992379951453, 2.274617397275522)
../input/studentengagement/Student-engagement-dataset/Engaged/frustrated/0375.jpg
Angle:  (-161.32576176209056, 8.464223022683747, -2.869348531172143)
Angle:  (-161.32576176209056, 8.464223022683747, -2.869348531172143)
../input/studentengagement/Student-engagement-dataset/Engaged/frustrated/0285.jpg
Angle:  (173.4722010413148, -0.05571447804324895, 9.891281374710033)
Angle:  (173.4722010413148, -0.05571447804324895, 9.891281374710033)
../input/studentengagement/Student-engagement-dataset/Engaged/frustrated/0074.jpg
Angle:  (-172.1664120667452, 3.7074380875274944, 4.88889779031285)
Angle:  (-172.1664120667452, 3.7074380875274944, 4.88889779031285)
../input/studentengagement/Student-engagement-dataset/Engaged/frustrated/0077.jpg
Angle:  (-171.2765605403431, 5.213980329612163, 4.834513829282902)
Angle:  (-171.2765605403431, 5.213980329612163, 4.8345138

In [12]:
print(my_list1)
print(drow)
print(file_path)
print(emotion)
print(label)

[1, 0, 1, 1, 0, 1, 0, 0, 0, None, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 0, 1, None, 0, 0, 0, None, 0, 0, 0, None, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, None, 0, 0, 0, None, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, None, 0, 1, 0, 0, 0, 0, None, 0, 0, 0, 0, None, 0, 0, 0, 0, None, 0, None, None, 0, 0, 0, 0, 0, None, None, 0, None, 0, 0, 0, 0, 0, 0, 0, None, 0, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 0, 0, 1, 0, 1, 0, None, 0, 0, 0, 1, None, 0, 0, None, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, None, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, None, 0, 0, 0, 0, 0, None, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, None, 0, 1, 0, 0, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, None, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, None, None, 0, None, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, None, 0, 0, 

In [13]:
df=pd.DataFrame()
df['image']=file_path
df['head_pose'] = my_list1
df['drowsiness'] = drow
df['emotion'] = emotion
df['label'] = label

In [14]:
df.head()

,image,head_pose,drowsiness,emotion,label
0,../input/studentengagement/Student-engagement-...,1.0,Active!!,Frustrated,0.40
1,../input/studentengagement/Student-engagement-...,0.0,Active!!,Frustrated,0.25
2,../input/studentengagement/Student-engagement-...,1.0,Active!!,Frustrated,0.40
3,../input/studentengagement/Student-engagement-...,1.0,Active!!,Frustrated,0.40
4,../input/studentengagement/Student-engagement-...,0.0,Active!!,Frustrated,0.25


In [15]:
df
df.emotion.unique()

array(['Frustrated', 'Confused'], dtype=object)